In [18]:
# MAIN BLOCK OF CODE
from datetime import * 
import pandas as pd
import openpyxl as op

# Reads excel file
df = pd.read_excel("RunLog.xlsx") 
user_action = input("Would you like to add data, edit data, delete data, or exit app? a/e/d/exit ")

# Add new row of data
def input_data():
    df = pd.read_excel("RunLog.xlsx") 
    add_data = "y"

    while add_data == "y":
        # User inputs with error handling
        date_str = input("Enter date (M/D/YYYY): ")
        try:
            # Convert date string into date object (has time)
            datetime_object = datetime.strptime(date_str, "%m/%d/%Y")

            # Extract just the date part to get a pure date object
            date = datetime_object.date()
        except ValueError:
            date_str = input("ERROR! Enter  valid date (M/D/YYYY): ")
            datetime_object = datetime.strptime(date_str, "%m/%d/%Y") 
            date = datetime_object.date()

        run_type = input("Enter run as \"road\" or \"trail\": ")
        if run_type != "road" or run_type != "trail":
            run_type = input("ERROR! Enter run as \"road\" or \"trail\": ")

        try:
            hours = int(input("Enter number of hours: "))
        except ValueError:
             hours = int(input("ERROR! Enter number of hours: "))

        try:
            minutes = int(input("Enter number of minutes: "))
        except ValueError:
            minutes = int(input("ERROR! Enter number of minutes: "))

        try:
            seconds = int(input("Enter number of seconds: "))
        except ValueError:
            seconds = int(input("ERROR! Enter number of seconds: "))

        try:
            miles = float(input("Enter number of miles: "))
        except ValueError:
             miles = float(input("ERROR! Enter number of miles: "))

        # Calculating pace
        convert_seconds = seconds / 60
        convert_hours = hours * 60
        total_minutes = minutes + convert_seconds + convert_hours
        pace = total_minutes / miles

        # Creating new row of data based on user inputs
        new_row = pd.DataFrame({'Date': [date], 'Run Type': [run_type], 'Hours': [hours], 'Minutes': [minutes], 'Seconds': [seconds], 'Miles': [miles], 'Pace':[pace]})

        # Adding new row 
        df = pd.concat([df, new_row], ignore_index=True)

        # Sends new row to excel spreadsheet
        df.to_excel("RunLog.xlsx", index=False)

        # Asking if user wants to continue adding more data
        add_data = input("Would you like to input data? y/n ")
        if add_data == "y":
            continue 
        else: 
            break 
    display(df)

# Edit data
def edit_the_data():
    df = pd.read_excel("RunLog.xlsx") 
    edit_data = "y" 

    while edit_data == "y":
        # User inputting what row and column to make edits in 
        index_input = int(input("Enter index to edit: "))
        select_column = input("Enter column name to edit: ")
       
        # Depending on column, will convert user input to correct data type
        if select_column == "Date":
            user_edit = input("Enter new data: ")
            
            # Convert date string into date object (has time)
            datetime_object = datetime.strptime(user_edit, "%m/%d/%Y")

            # Extract just the date part to get a pure date object
            date = datetime_object.date()

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = date
        elif select_column == "Run Type":
            user_edit = input("Enter new data: ")

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = user_edit
        elif select_column == "Miles":
            df.at[index_input, 'Miles'] = None
            user_edit = float(input("Enter new data: "))

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = user_edit

            # Recalculating pace
            convert_seconds = df.loc[index_input].at["Seconds"] / 60
            convert_hours = df.loc[index_input].at["Hours"] * 60
            total_minutes = df.loc[index_input].at["Minutes"] + convert_seconds + convert_hours
            pace = total_minutes / df.loc[index_input].at["Miles"]

            df.at[index_input, "Pace"] = pace

        else:
            user_edit = int(input("Enter new data: "))

            # Changes the data at the selected index and column
            df.at[index_input, select_column] = user_edit

            # Recalculating pace
            convert_seconds = df.loc[index_input].at["Seconds"] / 60
            convert_hours = df.loc[index_input].at["Hours"] * 60
            total_minutes = df.loc[index_input].at["Minutes"] + convert_seconds + convert_hours
            pace = total_minutes / df.loc[index_input].at["Miles"]

            df.at[index_input, "Pace"] = pace

        # Sends edit to excel spreadsheet
        df.to_excel("RunLog.xlsx", index=False)
        
        # Asking if user wants to continue editing data
        edit_data = input("Would you like to edit more data? y/n ")
        if edit_data == "y":
            continue
        else:
            break
    display(df)

# Delete row
def delete_row():
    df = pd.read_excel("RunLog.xlsx") 
    delete_data = "y"

    while delete_data == "y":
        # User inputting what row to delete
        index_input = int(input("Enter index to delete: "))

        # Delete row base on index
        df.drop([index_input], inplace=True)

        # Sends edit to excel spreadsheet
        df.to_excel("RunLog.xlsx", index=False)
        
        # Asking if user wants to continue to delete more rows of data
        delete_data = input("Would you like to delete another row? y/n ")
        if delete_data == "y": 
            continue
        else:
            break
    display(df)

# Base on user's choice, will either add, edit, delete, or show the table if exiting app
if user_action == "a":
    input_data()
if user_action == "e":
    edit_the_data()
if user_action == "d":
    delete_row()
if user_action == "exit":
    display(df)




,Date,Run Type,Hours,Minutes,Seconds,Miles,Pace
0,2024-01-01,road,1,1,1,1,61.016667
1,2024-01-02,trail,1,2,2,2,31.016667
2,2024-01-03,road,3,2,3,12,15.170833
3,2024-01-04,trail,4,4,4,4,61.016667
4,2024-01-05,road,3,3,3,10,18.305000


In [16]:
# MIAN CODE FOR CALCULATIONS 
# Temporary utilizing a different excel spreadsheet with a whole months worth of inputted running data

import pandas as pd

df = pd.read_excel("RunLog_Test.xlsx")
df.set_index('Date', inplace=True)
# df['Date']=pd.to_datetime(df['Date'])

road_miles = 0.0
trail_miles = 0.0

month_input = input("Enter month: ")
# df = df[df["Date"].dt.month == month_input]
# df = df[(df['Date']).dt.strftime('%B') == month_input]
# df=df[((df['Date'].dt.month==month_input))]
# df[df['Dates'].month == 11]['Dates']

runtype_group = df.groupby('Run Type')
monthly = runtype_group.resample('ME')
# print(monthly)

# FIXME: Add a way to interate base on month
# Iterates through each row and adds the total road and trail run miles
for index, row in df.iterrows():
    # while index == (df.loc[(df['Date'].dt.month==month_input)]):
    if df.at[index, "Run Type"] == "road":
        road_miles += df.at[index, "Miles"]
    if df.at[index, "Run Type"] == "trail":
        trail_miles += df.at[index, "Miles"]

# Calculations
weekly_road_average = road_miles / 4
weekly_trail_average = trail_miles / 4
weekly_total_average = weekly_road_average + weekly_trail_average # Weekly average for the month

print("Weekly Road Run Average:", weekly_road_average)
print("Weekly Trail Run Average:", weekly_trail_average)
print(f"Weekly Total Average for {month_input}: {weekly_total_average}")


Weekly Road Run Average: 13.4875
Weekly Trail Run Average: 18.725
Weekly Total Average for January: 32.212500000000006


In [ ]:
# NOTE: OLD CODE
# ADDING NEW DATA
# FIXME: Stop having new index columns added 
# FIXED: df.to_excel("RunLog.xlsx", index=False)

import pandas as pd
# import openpyxl as op

# Reads excel file
df = pd.read_excel("RunLog.xlsx") 

add_data = input("Would you like to input data? y/n ")

while add_data == "y":
    # User inputs
    date = input("Enter date (M/D/YYYY): ")
    run_type = input("Enter road or trail: ")
    hours = int(input("Enter number of hours: "))
    minutes = int(input("Enter number of minutes: "))
    seconds = int(input("Enter number of seconds: "))
    miles = float(input("Enter number of miles: "))

    # Creating new row of data based on user inputs
    new_row = pd.DataFrame({'Date': [date], 'Run Type': [run_type], 'Hours': [hours], 'Minutes': [minutes], 'Seconds': [seconds], 'Miles': [miles]})
    
    # Adding new row 
    df = pd.concat([df, new_row], ignore_index=True)
   
    # Sends new row to excel spreadsheet
    df.to_excel("RunLog.xlsx", index=False)

    # Asking if user wants to continue adding more data
    add_data = input("Would you like to input data? y/n ")
    if add_data == "y":
        continue 
    else: 
        break 
print(df)

In [ ]:
# CH with using datetime for dates
from datetime import datetime
import pandas as pd
import openpyxl as op

# Reads excel file
df = pd.read_excel("CH.xlsx") 

add_data = input("Would you like to input data? y/n ")

while add_data == "y":
    # User inputs
    date_str = input("Enter date (M/D/YYYY): ")
    run_type = input("Enter road or trail: ")
    hours = int(input("Enter number of hours: "))
    minutes = int(input("Enter number of minutes: "))
    seconds = int(input("Enter number of seconds: "))
    miles = float(input("Enter number of miles: "))

    # convert date string into date object (has time)
    datetime_object = datetime.strptime(date_str, "%m/%d/%Y")

    # Extract just the date part to get a pure date object
    date = datetime_object.date()

    # Creating new row of data based on user inputs
    new_row = pd.DataFrame({'Date': [date], 'Run Type': [run_type], 'Hours': [hours], 'Minutes': [minutes], 'Seconds': [seconds], 'Miles': [miles]})
    
    print(new_row)

    # Adding new row 
    df = pd.concat([df, new_row], ignore_index=True)
   
    # Sends new row to excel spreadsheet
    df.to_excel("CH.xlsx", index=False)

    # Asking if user wants to continue adding more data
    add_data = input("Would you like to input data? y/n ")
    if add_data == "y":
        continue 
    else: 
        break 
print(df)

In [19]:
display(pd.read_excel("CH.xlsx")) 

,Date,Run Type,Hours,Minutes,Seconds,Miles
0,2024-12-04,road,5,12,0,6.5


In [ ]:
# NOTE: OLD EDITING DATA 
# FIXME: Stop having new index columns added 
# NOTE: HAS BEEN FIXED 

import pandas as pd

df = pd.read_excel("RunLog.xlsx") 
edit_data = input("Would you like to edit data? y/n ")

while edit_data == "y":
    # User inputting what row and column to make edits in
    index_input = int(input("Enter index to edit: "))
    select_column = input("Enter column name to edit: ")
    
    # Depending on column, will convert user input to correct data type
    if select_column == "Date" or select_column == "Run Type":
        user_edit = input("Enter new data: ")
    elif select_column == "Miles":
        df.at[index_input, 'Miles'] = None
        user_edit = float(input("Enter new data: "))
    else:
        user_edit = int(input("Enter new data: "))
     
    # Changes the data at the selected index and column
    df.at[index_input, select_column] = user_edit

    # Sends edit to excel spreadsheet
    df.to_excel("RunLog.xlsx", index=False)
    
    # Asking if user wants to continue editing data
    edit_data = input("Would you like to edit data? y/n ")
    if edit_data == "y":
        continue
    else:
        break
print(df)


In [ ]:
# NOTE: OLD CALCULATIONS 
# For now, utilizing a different excel spreadsheet with a whole months worth of inputted running data

import pandas as pd

df = pd.read_excel("RunLog_Test.xlsx") 

road_miles = 0.0
trail_miles = 0.0
counter = 0

# Iterates through each row and adds the total road and trail run miles
for index, row in df.iterrows():
    if counter <= 27:
        if df.at[index, "Run Type"] == "road":
            road_miles += df.at[index, "Miles"]
            counter += 1
        if df.at[index, "Run Type"] == "trail":
            trail_miles += df.at[index, "Miles"]
            counter += 1
    elif counter > 28:
        counter = 0


# Calculations
weekly_road_average = road_miles / 4
weekly_trail_average = trail_miles / 4
weekly_total_average = weekly_road_average + weekly_trail_average

print("Weekly Road Run Average:", weekly_road_average)
print("Weekly Trail Run Average:", weekly_trail_average)
print("Weekly Total Average:", weekly_total_average)



In [19]:
# CH
df = pd.read_excel("log_for_avg.xlsx") 
# df = pd.read_excel("RunLog_Test.xlsx") 

# Set 'Date' as the index
df.set_index('Date', inplace=True)
display(df)

runtype_group = df.groupby('Run Type')
weekly = runtype_group.resample('W')
weekly_miles_mean = weekly.mean()['Miles']

# sort (re-index) by date
weekly_miles_mean = weekly_miles_mean.sort_index(level='Date')

print(weekly)
print(weekly_miles_mean) # dataframe indexed by time



,Run Type,Hours,Minutes,Seconds,Miles
Date,,,,,
2024-12-04,road,5,12,0,6.5
2024-12-11,trail,4,30,0,7.0
2024-12-18,road,6,15,0,6.0
2024-12-25,trail,3,45,0,7.5
2025-01-01,road,5,20,0,6.8


DatetimeIndexResamplerGroupby [freq=<Week: weekday=6>, axis=0, closed=right, label=right, convention=e, origin=start_day]
Run Type  Date      
road      2024-12-08    6.5
          2024-12-15    NaN
trail     2024-12-15    7.0
road      2024-12-22    6.0
trail     2024-12-22    NaN
road      2024-12-29    NaN
trail     2024-12-29    7.5
road      2025-01-05    6.8
Name: Miles, dtype: float64


In [20]:
# dropna() remove NaN rows
table_as_nested_list = list(weekly_miles_mean.dropna().items())
# items() give us a nested structure
table_as_nested_list

[(('road', Timestamp('2024-12-08 00:00:00')), 6.5),
 (('trail', Timestamp('2024-12-15 00:00:00')), 7.0),
 (('road', Timestamp('2024-12-22 00:00:00')), 6.0),
 (('trail', Timestamp('2024-12-29 00:00:00')), 7.5),
 (('road', Timestamp('2025-01-05 00:00:00')), 6.8)]

In [21]:
# unpack the type, date and miles and print
for (run_type, timestamp), miles in table_as_nested_list:
    date_only = timestamp.date()
    print(f"{date_only}: {run_type}: {miles} on avg.")

2024-12-08: road: 6.5 on avg.
2024-12-15: trail: 7.0 on avg.
2024-12-22: road: 6.0 on avg.
2024-12-29: trail: 7.5 on avg.
2025-01-05: road: 6.8 on avg.
